In [1]:
import json
import urllib2
import csv

In [2]:

task_ids = []
with open('id_fishing_points_task.json') as data_file:    
    data = json.load(data_file)
    for d in data:
        task_ids.append(d['id'])


In [6]:
task_ids = []
data = []
with open('id_fishing_points_task.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        task_ids.append(row['task__id'])
        d = {}
        d['info'] = {}
        d['id'] = row['task__id']
        d['info']= json.loads(row['task__info'])
        data.append(d)



In [7]:
print data[0]

{'info': {u'mmsi': u'224082260', u'year': u'2015', u'vesselType': u'Longliner', u'month': u'1'}, 'id': '6031'}


In [8]:
for d in data:
    t = d['id']
    year = d['info']['year']
    month = d['info']['month']
    mmsi = d['info']['mmsi']
    
    
    url = "http://storage.googleapis.com/gfw-crowd/"+str(mmsi)+"_"+str(year)+"_"+str(month)+".json"
    response = urllib2.urlopen(url)
    input_data = json.loads(response.read())

        
    sogs = input_data['sogs']
    timestamps = input_data['timestamps']
    lons = input_data['lons']
    lats = input_data['lats']
    
    f = open("jsons/"+str(t)+".json",'rU')
    contents = f.read()
    f.close()
    results = json.loads(contents)
    headers = []
    fishingArrays = []
    for response in results:
        user_id = response['user_id']
        if response['info'] != "hello":
            r = json.loads(response['info'])
            confidence = r['confidence']
            if confidence != 'no_confidence':
                vesselType = r['vesselType']
                fishingArrayString = r['fishingArrayString']
                fishingArrays.append([int(fishingArrayString[i]) for i in range(len(fishingArrayString))])
                headers.append("user_id: "+str(user_id)+", "+confidence)

    if len(headers)>0:
        with open(vesselType+"_"+str(mmsi)+"_"+str(year)+"_"+str(month)+'.csv', 'wb') as f:
            writer = csv.writer(f)
            theheader = ['mmsi','timestamp','speed','lat','lon'] + headers
            writer.writerow(theheader)        
            for i in range(len(lats)):
                row = [mmsi, timestamps[i],sogs[i],lats[i],lons[i]]
                for j in range(len(headers)):
                    row += [fishingArrays[j][i]]
                writer.writerow(row)



IOError: [Errno 2] No such file or directory: 'jsons/6367.json'